In [5]:
import numpy as np
import pandas as pd

## Handling Missing Data

In [3]:
vals1 = np.array([1,None,3,4,np.nan])
vals1

array([1, None, 3, 4, nan], dtype=object)

In [4]:
pd.Series(vals1) #Numpy treat the elements as objects

0       1
1    None
2       3
3       4
4     NaN
dtype: object

In [5]:
pd.Series([1, np.nan, 2, None]) #Whereas pandas treats them as float64

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [6]:
data = pd.Series([1, np.nan, 'hello', None])
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [7]:
data[data.notnull()]

0        1
2    hello
dtype: object

In [8]:
data.dropna()

0        1
2    hello
dtype: object

In [11]:
df = pd.DataFrame([[1, np.nan, 2],
                   [2, 3, 5],
                   [np.nan, 4, 6]])
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [14]:
df.dropna(axis='columns', how='all') #Over columns where all values are NaN

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [15]:
df.dropna(axis='rows', thresh=3) #Keep minimum 3 non-null value per row/col

,0,1,2,3
1,2.0,3.0,5,NaN


In [18]:
#Fill out NaN
df.fillna(0)

,0,1,2,3
0,1.0,0.0,2,0.0
1,2.0,3.0,5,0.0
2,0.0,4.0,6,0.0


In [20]:
#Forward-fill: fill out with the previous value
data.fillna(method='ffill')

0        1
1        1
2    hello
3    hello
dtype: object

In [22]:
data.fillna(method='bfill') #back-fill

0        1
1    hello
2    hello
3     None
dtype: object

In [23]:
df.fillna(method='ffill', axis=1) #Say where it will apply

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


## Hierarchical Indexing

In [30]:
#Multi dimensional structures
index = [('California',2000),('California',2010),
         ('New York',2000),('New York',2010),
         ('Texas',2000),('Texas',2010)]
populations = [1000, 2000, 3000, 4000, 5000, 6000]
pop = pd.Series(populations, index=index)

In [31]:
index = pd.MultiIndex.from_tuples(index) 
print(index)
pop = pop.reindex(index)
pop

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])


California  2000    1000
            2010    2000
New York    2000    3000
            2010    4000
Texas       2000    5000
            2010    6000
dtype: int64

In [45]:
pop[:,2010]

California    2000
New York      4000
Texas         6000
dtype: int64

In [46]:
pop_df = pd.DataFrame({'total':pop, 
                       'under18': [100, 200, 300, 400, 500, 600]})
pop_df

total  under18
California 2000   1000      100
           2010   2000      200
New York   2000   3000      300
           2010   4000      400
Texas      2000   5000      500
           2010   6000      600

In [48]:
#Name index
pop_df.index.names = ['state','year']
pop_df

total  under18
state      year                
California 2000   1000      100
           2010   2000      200
New York   2000   3000      300
           2010   4000      400
Texas      2000   5000      500
           2010   6000      600

In [49]:
#MultiIndex for Columns

index = pd.MultiIndex.from_product([[2013,2014],[1,2]],
                                   names=['year','visit'])
columns = pd.MultiIndex.from_product([['Bob','Guido','Sue'],['HR','Temp']],
                                     names = ['subject','type'])
#mock some data
data = np.round(np.random.randn(4,6),1)
data[:,::2] *= 10
data +=37

health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      36.0  36.7  28.0  36.4  38.0  38.8
     2      31.0  36.1  26.0  38.2  31.0  38.4
2014 1      30.0  38.6  41.0  36.1  33.0  37.6
     2      39.0  36.8  33.0  36.6  38.0  38.1

In [50]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      28.0  36.4
     2      26.0  38.2
2014 1      41.0  36.1
     2      33.0  36.6

In [57]:
#Slicing the data
health_data['Guido','HR']
health_data.iloc[:3,:3]
#health_data.loc[:,('Bob','HR')]

subject      Bob       Guido
type          HR  Temp    HR
year visit                  
2013 1      36.0  36.7  28.0
     2      31.0  36.1  26.0
2014 1      30.0  38.6  41.0

In [58]:
idx = pd.IndexSlice
health_data.loc[idx[:,1], idx[:,'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,36.0,28.0,38.0
2014,1,30.0,41.0,33.0


In [60]:
#To correctly slice data in indexes, they must be sorted
index = pd.MultiIndex.from_product([['a','c','b'],[1,2]])
data = pd.Series(np.random.rand(6),index=index)
data.index.names=['char','int']
data

char  int
a     1      0.145186
      2      0.073298
c     1      0.387533
      2      0.365087
b     1      0.311546
      2      0.885741
dtype: float64

In [62]:
data = data.sort_index()
data

char  int
a     1      0.145186
      2      0.073298
b     1      0.311546
      2      0.885741
c     1      0.387533
      2      0.365087
dtype: float64

In [63]:
#Stacking and unstacking indices
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,1000,3000,5000
2010,2000,4000,6000


In [64]:
pop.unstack(level=1)

year,2000,2010
state,,
California,1000,2000
New York,3000,4000
Texas,5000,6000


In [65]:
pop.unstack().stack()

state       year
California  2000    1000
            2010    2000
New York    2000    3000
            2010    4000
Texas       2000    5000
            2010    6000
dtype: int64

In [66]:
# You can rearrange the index or create it from the columns of a dataframe
pop_flat = pop.reset_index(name='population') 
pop_flat #around the values of population it creates a new index

,state,year,population
0,California,2000,1000
1,California,2010,2000
2,New York,2000,3000
3,New York,2010,4000
4,Texas,2000,5000
5,Texas,2010,6000


In [68]:
pop_flat.set_index(['state','year'])

population
state      year            
California 2000        1000
           2010        2000
New York   2000        3000
           2010        4000
Texas      2000        5000
           2010        6000

In [70]:
#You can aggregate data using indexes
data_mean = health_data.mean(level='year')
data_mean

subject   Bob       Guido          Sue       
type       HR  Temp    HR   Temp    HR   Temp
year                                         
2013     33.5  36.4  27.0  37.30  34.5  38.60
2014     34.5  37.7  37.0  36.35  35.5  37.85

In [71]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,31.666667,37.433333
2014,35.666667,37.300000


## Combining Datasets

#### Concat and Append

In [8]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


In [17]:
#keeps duplicated indexes
x = make_df('AB',[0,1])
y = make_df('AB',[2,3])
print(pd.concat([x,y]))         #Append in rows
print(pd.concat([x,y], axis=1)) #Append in columns
y.index = x.index               #make duplicate indices
print(x); print(y); print(pd.concat([x,y]))
pd.concat([x,y],keys=['x','y'])

    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3
     A    B    A    B
0   A0   B0  NaN  NaN
1   A1   B1  NaN  NaN
2  NaN  NaN   A2   B2
3  NaN  NaN   A3   B3
    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

In [18]:
x.append(y)

,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


#### Merge and Join

In [19]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
'salary': [70000, 80000, 120000, 90000]})

In [20]:
#merge on employee in df1 and name on df3, take out name as it is repeated
pd.merge(df1,df3,left_on='employee',right_on='name').drop('name',axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


In [21]:
pop = pd.read_csv('https://github.com/jakevdp/data-USstates/raw/master/state-population.csv')
areas = pd.read_csv('https://github.com/jakevdp/data-USstates/raw/master/state-areas.csv')
abbrevs = pd.read_csv('https://github.com/jakevdp/data-USstates/raw/master/state-abbrevs.csv')

In [22]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [23]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [24]:
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [59]:
#Rank US states and territories by their 2010 population density
#data2010 = pop[pop.year == 2010]
data2010 = pd.merge(pop[(pop.year == 2010) & (pop.ages == 'total')],abbrevs,how='outer',
                   left_on='state/region', right_on='abbreviation').drop('abbreviation',axis=1)

In [60]:
data2010.isnull().any()

state/region    False
ages            False
year            False
population      False
state            True
dtype: bool

In [61]:
data2010.loc[data2010.state.isnull(),'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [62]:
data2010.loc[data2010['state/region'] == 'PR', 'state'] = 'Puerto Rico'
data2010.loc[data2010['state/region'] == 'USA', 'state'] = 'United States'
data2010.isnull().any()

state/region    False
ages            False
year            False
population      False
state           False
dtype: bool

In [63]:
data2010 = pd.merge(data2010, areas, on = 'state', how = 'left')
data2010.isnull().any()

state/region     False
ages             False
year             False
population       False
state            False
area (sq. mi)     True
dtype: bool

In [64]:
data2010.state[data2010['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [66]:
data2010.dropna(inplace=True)
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,total,2010,4785570.0,Alabama,52423.0
1,AK,total,2010,713868.0,Alaska,656425.0
2,AZ,total,2010,6408790.0,Arizona,114006.0
3,AR,total,2010,2922280.0,Arkansas,53182.0
4,CA,total,2010,37333601.0,California,163707.0


In [67]:
data2010.set_index('state',inplace=True)

In [68]:
density = data2010.population / data2010['area (sq. mi)']
density.sort_values(ascending=False,inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [69]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64